In [2]:
from scipy.special import factorial
from scipy.stats import binom
import pandas as pd
import numpy as np
import math

In [3]:
#function for calculating the total cost. total purchase and the derivative of f(B)/g(B) over Bi

def cal_tcost(a,b0,M,B,C,S):
    tmp=0.0
    for i in range(0,len(B)):
        x=a[i]*(B[i]-10.0)+b0[i]
        P=1/(1+math.exp(-x))
        for r in range(0,5):
            tmp+=binom.pmf(r,4,P)*B[i]*C[i,r]*M[i]
    return tmp

def cal_tsr(a,b0,M,B,S):
    tmp=0.0
    for i in range(0,len(B)):
        x=a[i]*(B[i]-10.0)+b0[i]
        P=1/(1+math.exp(-x))
        for r in range(0,5):
            tmp+=binom.pmf(r,4,P)*S[i,r]*M[i]
    return tmp

#calculate the derivative over Bj, return a float
def calc_fgderive_entry(a,b0,M,B,C,S,j):
    S_den=cal_tsr(a,b0,M,B,S)
    C_num=cal_tcost(a,b0,M,B,C,S)
    coe1=M[j]/S_den
    coe2=C_num/(S_den**2)*a[j]*M[j]
    x=a[j]*(B[j]-10.0)+b0[j]
    P=1/(1+math.exp(-x))
    sum1=0.0
    sum2=0.0
   
    for r in range(0,5):
        sum1+=1+a[j]*B[j]*(r-1-4*P)*C[j,r]*binom.pmf(r,4,P)
        sum2+=(r-1-4*P)*S[j,r]*binom.pmf(r,4,P)
        tmp=sum1*coe1-sum2*coe2
    return tmp
    

In [4]:
#read in data and make M,C,S,b0 arrays
df=pd.read_csv("predict_proba_averaged.csv")
df_xy=pd.read_csv("bi0.csv")

Insure=["Y","N","unknown"]
Ncars=[1,2,3]
Ndrivers=[1,2]
Marital=["S","M"]
Count=0

C=np.zeros((36,5))
S=np.zeros((36,5))
M=np.zeros(36)
b0=np.zeros(36)

for i in Insure:
    for j in Ncars:
        for n in Ndrivers:
            for m in Marital:
                C[Count]=df.loc[(df["Currently Insured"]==i) & (df["Number of Vehicles"]==j) & (df["Number of Drivers"]==n) & (df["Marital Status"]==m)]['ave_Click_pred'].to_numpy()
                S[Count]=df.loc[(df["Currently Insured"]==i) & (df["Number of Vehicles"]==j) & (df["Number of Drivers"]==n) & (df["Marital Status"]==m)]['ave_Purchase_pred'].to_numpy()
                M[Count]=df_xy.loc[(df_xy["Currently Insured"]==i) & (df_xy["Number of Vehicles"]==j) & (df_xy["Number of Drivers"]==n) & (df_xy["Marital Status"]==m)]['Mi'].to_numpy()[0]
                b0[Count]=df_xy.loc[(df_xy["Currently Insured"]==i) & (df_xy["Number of Vehicles"]==j) & (df_xy["Number of Drivers"]==n) & (df_xy["Marital Status"]==m)]['bi0'].to_numpy()[0]
                #print(type(df_xy.loc[(df["Currently Insured"]==i) & (df["Number of Vehicles"]==j) & (df["Number of Drivers"]==n) & (df["Marital Status"]==m)]['bi0'].to_numpy()[0]))
                Count+=1

#drop the type that doesn't exist: unknown, 3 cars, 2 drivers, single, index=34
#np arrays are mutable 
C=np.delete(C,34,0)
S=np.delete(S,34,0)
M=np.delete(M,34)
b0=np.delete(b0,34)

In [5]:
#Basic gradient descent, stop when purchase rate falls below 4%
def GD_basic(eps,step,Nitr,B):
    for i in range(0,Nitr):
        B_pri=np.zeros(len(B))
        for j in range(0,len(M)):
            tmp=B[j]-step*calc_fgderive_entry(a,b0,M,B,C,S,j)
            if tmp>0:
                B_pri[j]=tmp
    
        tsr=cal_tsr(a,b0,M,B,S)
        tsr_pri=cal_tsr(a,b0,M,B_pri,S)
        if tsr_pri>0.04:
            tcost=cal_tcost(a,b0,M,B,C,S)
            tcost_pri=cal_tcost(a,b0,M,B_pri,C,S)
            if (tcost/tsr-tcost_pri/tsr_pri)>=eps:
                B=B_pri
            else:
                print("stop at interation "+str(i)+"\n Cause:reach the smallest f(B)/g(B) difference "+str(tcost/tsr-tcost_pri/tsr_pri))
                break
        else:
            print("stop at interation "+str(i)+"\n Cause: purcahse rate is below 4%")
            break
        if i==(Nitr-1):
            print("run out of itrations")

    print(B)
    print(cal_tsr(a,b0,M,B,S))
    print(cal_tcost(a,b0,M,B_pri,C,S)/cal_tsr(a,b0,M,B,S))
    return B        

In [6]:
#Josh's modified Projected gradient descent algorithm
def GD_Josh(eps,step,Nitr,B):
    for i in range(0,Nitr):
        B_pri=np.zeros(len(B))
        for j in range(0,len(M)):
            tmp=B[j]-step*calc_fgderive_entry(a,b0,M,B,C,S,j)
            if tmp>0:
                B_pri[j]=tmp
    
        tsr=cal_tsr(a,b0,M,B,S)
        tsr_pri=cal_tsr(a,b0,M,B_pri,S)
        if tsr_pri>0.04:
            tcost=cal_tcost(a,b0,M,B,C,S)
            tcost_pri=cal_tcost(a,b0,M,B_pri,C,S)
            if (tcost/tsr-tcost_pri/tsr_pri)>=eps:
                B=B_pri
            else:
                print("stop at interation "+str(i)+"\n Cause:reach the smallest f(B)/g(B) difference "+str(tcost/tsr-tcost_pri/tsr_pri))
                break
        elif (tcost/tsr-tcost_pri/tsr_pri)>=eps:
            L=0.0
            for j in range(0,len(M)):
                L+=calc_fgderive_entry(a,b0,M,B,C,S,j)**2
            L=math.sqrt(L)*step
            jstar=-1
            mstar=-1
            Hstar=1000
            for j in range(0,len(M)):
                for m in [0,1]:
                    B_pri_copy=B_pri.copy()
                    if B_pri_copy[j]==0 and m==0:
                        continue
                    B_pri_copy[j]=B_pri_copy[j]+(-1)**m*L
                    cpc_pri=cal_tcost(a,b0,M,B_pri_copy,C,S)/cal_tsr(a,b0,M,B_pri_copy,S)
                    cpc=cal_tcost(a,b0,M,B,C,S)/cal_tsr(a,b0,M,B,S)
                    if cal_tsr(a,b0,M,B_pri_copy,S)>0.04 and cpc_pri< min(cpc-eps,Hstar):
                        jstar=j
                        mstar=m
                        Hstar=cpc_pri
            if jstar>=0 and mstar>=0:
                B[jstar]=B[jstar]+(-1)**mstar*L
        
        
        if i==(Nitr-1):
            print("run out of itrations")

    print(cal_tsr(a,b0,M,B,S))
    print(cal_tcost(a,b0,M,B_pri,C,S)/cal_tsr(a,b0,M,B,S))
    return B 

In [7]:
#random assign the initial B
B=np.zeros(len(M))
a=np.full(len(M),-0.5)
while cal_tsr(a,b0,M,B,S)<0.04:
    for i in range(0,len(B)):
        B[i]=np.random.randint(1,20)
        
print(B)
print(cal_tsr(a,b0,M,B,S))
print(cal_tcost(a,b0,M,B,C,S)/cal_tsr(a,b0,M,B,S))

[ 9. 11.  5.  5. 19.  5. 19.  5. 14.  4. 10. 11. 15.  7. 10.  8.  5.  6.
  3.  8. 19.  1. 14. 15.  9. 17.  2. 19.  8. 12.  7. 18. 13. 13. 13.]
0.11107308822459752
26.651590643244504


In [8]:
#set the Nitr,eps(smallest f(B)/g(B) difference) and step
eps=0.0001
step=0.05
Nitr=100

In [9]:
#set a and run the algorithm 
a=np.full(len(M),-0.5)
B_m0p5=GD_Josh(eps,step,Nitr,B)

run out of itrations
0.040134850697029115
6.839122704308893


In [11]:
print(B)
print(B_m0p5)

[ 9. 11.  5.  5. 19.  5. 19.  5. 14.  4. 10. 11. 15.  7. 10.  8.  5.  6.
  3.  8. 19.  1. 14. 15.  9. 17.  2. 19.  8. 12.  7. 18. 13. 13. 13.]
[0.38074609 2.49209962 0.         0.         9.27578275 0.
 8.17833388 0.         5.44512294 0.         1.60116528 2.33441408
 7.03822842 0.         1.39328219 0.         0.         0.
 0.         0.         8.57038614 0.         4.67181    4.50718349
 0.98366991 7.32071085 0.         7.87317344 0.         3.75965121
 0.         7.5600943  4.7852389  3.88736753 4.2919003 ]


In [12]:
a=np.full(len(M),-0.1)
B_m0p1=GD_Josh(eps,step,Nitr,B)

run out of itrations
0.06987139452409354
6.747683985274571


In [13]:
print(B_m0p1)

[ 0.14336533  2.50659933  0.          0.         10.5889834   0.
  9.39190185  0.          5.66597617  0.          1.65393609  2.44172612
  6.4449842   0.          1.29829269  0.          0.          0.
  0.          0.          9.93420132  0.          4.93478555  4.84599
  0.50801096  7.42885243  0.          9.34271406  0.          3.64869201
  0.          7.96911908  4.96942483  4.01349144  4.33684619]


In [14]:
a=np.full(len(M),-1.0)
B_m1=GD_Josh(eps,step,Nitr,B)

run out of itrations
0.04287500584364062
7.801189312782474


In [15]:
print(B_m1)

[ 0.93143718  2.96788179  0.          0.          8.45223982  0.
  7.60711674  0.          5.72706178  0.          1.99097063  2.7034586
  6.03493753  0.          1.89647945  0.23993407  0.          0.
  0.          0.         11.51497351  0.          4.95534855  4.70912078
  1.62033444  9.76005818  0.          9.21661132  0.50657766  4.33177838
  0.          8.75685299  4.97184179  4.27315644  4.78376393]


In [ ]:
print